# L01: First Agent Implementation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/agentic-artificial-intelligence/blob/main/L01_Introduction_Agentic_AI/notebooks/L01_first_agent.ipynb)

**Week 1 - Introduction to Agentic AI**

## Learning Objectives
- Implement a basic ReAct agent from scratch
- Understand the Thought-Action-Observation loop
- Compare agent behavior vs standard LLM inference

## Prerequisites
- Python 3.11+
- OpenAI API key or Anthropic API key
- Basic understanding of LLM APIs

## 1. Setup and Configuration (10 min)

In [ ]:
# Colab setup
import sys
if 'google.colab' in sys.modules:
    !pip install -q openai langchain langchain-openai python-dotenv
    from google.colab import userdata
    import os
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
else:
    # Local setup - install if needed
    # !pip install openai langchain langchain-openai python-dotenv
    pass

In [ ]:
import os
import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is set
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("Warning: OPENAI_API_KEY not found. Set it in .env file.")
else:
    print("API key loaded successfully.")

## 2. Understanding the ReAct Loop

The ReAct paradigm (Yao et al., 2023) interleaves:
- **Thought**: Reasoning about current state
- **Action**: External operation
- **Observation**: Feedback from environment

```
Trajectory: τ = (s₀, t₁, a₁, o₁, t₂, a₂, o₂, ...)
```

In [ ]:
@dataclass
class ThoughtActionObservation:
    """Single step in a ReAct trajectory."""
    thought: str
    action: str
    action_input: str
    observation: str

@dataclass
class AgentTrajectory:
    """Complete ReAct trajectory."""
    question: str
    steps: List[ThoughtActionObservation]
    final_answer: Optional[str] = None
    
    def __str__(self):
        output = f"Question: {self.question}\n\n"
        for i, step in enumerate(self.steps, 1):
            output += f"Step {i}:\n"
            output += f"  Thought: {step.thought}\n"
            output += f"  Action: {step.action}[{step.action_input}]\n"
            output += f"  Observation: {step.observation}\n\n"
        if self.final_answer:
            output += f"Final Answer: {self.final_answer}"
        return output

## 3. Define Tools

Our agent will have access to simple tools for demonstration.

In [ ]:
class SimpleTool:
    """Base class for agent tools."""
    name: str = "base_tool"
    description: str = "Base tool"
    
    def run(self, input_str: str) -> str:
        raise NotImplementedError

class SearchTool(SimpleTool):
    """Simulated search tool with predefined knowledge."""
    name = "Search"
    description = "Search for information. Input: search query."
    
    # Simulated knowledge base
    knowledge = {
        "capital of france": "Paris is the capital of France.",
        "capital of germany": "Berlin is the capital of Germany.",
        "react paper": "ReAct was published by Yao et al. at ICLR 2023.",
        "langchain": "LangChain is a framework for building LLM applications.",
        "python": "Python is a programming language created by Guido van Rossum.",
        "llm agent": "An LLM agent is an autonomous system that uses an LLM for reasoning.",
    }
    
    def run(self, input_str: str) -> str:
        query = input_str.lower().strip()
        for key, value in self.knowledge.items():
            if key in query:
                return value
        return f"No results found for: {input_str}"

class CalculatorTool(SimpleTool):
    """Simple calculator tool."""
    name = "Calculator"
    description = "Perform arithmetic. Input: math expression."
    
    def run(self, input_str: str) -> str:
        try:
            # Safe evaluation of simple math
            allowed = set('0123456789+-*/(). ')
            if all(c in allowed for c in input_str):
                result = eval(input_str)
                return str(result)
            return "Invalid expression"
        except Exception as e:
            return f"Error: {str(e)}"

class FinishTool(SimpleTool):
    """Tool to signal task completion."""
    name = "Finish"
    description = "Use when you have the final answer. Input: the answer."
    
    def run(self, input_str: str) -> str:
        return f"FINISHED: {input_str}"

# Initialize tools
TOOLS = {
    "Search": SearchTool(),
    "Calculator": CalculatorTool(),
    "Finish": FinishTool(),
}

print("Available tools:")
for name, tool in TOOLS.items():
    print(f"  - {name}: {tool.description}")

## 4. Build the ReAct Agent

Now we'll build a simple ReAct agent that:
1. Receives a question
2. Generates thoughts and selects actions
3. Executes actions and receives observations
4. Iterates until reaching an answer

In [ ]:
from openai import OpenAI

client = OpenAI()

REACT_PROMPT = """You are a ReAct agent that solves problems by thinking step-by-step and using tools.

Available tools:
{tools}

Use this format:
Thought: reason about what to do next
Action: tool_name[input]

When you have the final answer, use:
Thought: I now have the answer
Action: Finish[your answer]

Begin!

Question: {question}
{history}
"""

def format_tools(tools: Dict[str, SimpleTool]) -> str:
    """Format tools for prompt."""
    return "\n".join([f"- {name}: {tool.description}" for name, tool in tools.items()])

def format_history(steps: List[ThoughtActionObservation]) -> str:
    """Format previous steps for context."""
    if not steps:
        return ""
    history = ""
    for step in steps:
        history += f"\nThought: {step.thought}\n"
        history += f"Action: {step.action}[{step.action_input}]\n"
        history += f"Observation: {step.observation}\n"
    return history

def parse_action(response: str) -> tuple[str, str, str]:
    """Parse LLM response to extract thought and action."""
    lines = response.strip().split('\n')
    thought = ""
    action = ""
    action_input = ""
    
    for line in lines:
        if line.startswith('Thought:'):
            thought = line[8:].strip()
        elif line.startswith('Action:'):
            action_str = line[7:].strip()
            # Parse Action[input] format
            if '[' in action_str and ']' in action_str:
                action = action_str[:action_str.index('[')]
                action_input = action_str[action_str.index('[')+1:action_str.rindex(']')]
    
    return thought, action, action_input

In [ ]:
class ReActAgent:
    """Simple ReAct agent implementation."""
    
    def __init__(self, tools: Dict[str, SimpleTool], max_steps: int = 5):
        self.tools = tools
        self.max_steps = max_steps
    
    def run(self, question: str, verbose: bool = True) -> AgentTrajectory:
        """Run the agent on a question."""
        trajectory = AgentTrajectory(question=question, steps=[])
        
        for step_num in range(self.max_steps):
            # Build prompt with history
            prompt = REACT_PROMPT.format(
                tools=format_tools(self.tools),
                question=question,
                history=format_history(trajectory.steps)
            )
            
            # Get LLM response
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0
            )
            
            llm_output = response.choices[0].message.content
            thought, action, action_input = parse_action(llm_output)
            
            if verbose:
                print(f"\n--- Step {step_num + 1} ---")
                print(f"Thought: {thought}")
                print(f"Action: {action}[{action_input}]")
            
            # Execute action
            if action in self.tools:
                observation = self.tools[action].run(action_input)
            else:
                observation = f"Unknown action: {action}"
            
            if verbose:
                print(f"Observation: {observation}")
            
            # Record step
            step = ThoughtActionObservation(
                thought=thought,
                action=action,
                action_input=action_input,
                observation=observation
            )
            trajectory.steps.append(step)
            
            # Check if finished
            if action == "Finish":
                trajectory.final_answer = action_input
                break
        
        return trajectory

## 5. Test the Agent

In [ ]:
# Create agent
agent = ReActAgent(tools=TOOLS)

# Test question
question = "What is the capital of France?"
print(f"Question: {question}")
print("="*50)

trajectory = agent.run(question)

In [ ]:
# More complex question
question = "What is 25 * 4 + 100?"
print(f"\nQuestion: {question}")
print("="*50)

trajectory = agent.run(question)

## 6. Compare: LLM vs Agent

Let's compare the agent approach to direct LLM inference.

In [ ]:
def direct_llm(question: str) -> str:
    """Direct LLM inference without agent loop."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
        max_tokens=100,
        temperature=0
    )
    return response.choices[0].message.content

# Compare approaches
test_question = "What framework was created for building LLM applications?"

print("Direct LLM Response:")
print(direct_llm(test_question))

print("\n" + "="*50)
print("\nReAct Agent Response:")
trajectory = agent.run(test_question, verbose=True)

## 7. Key Takeaways

### What we learned:
1. **ReAct Loop**: Thought -> Action -> Observation
2. **Agent = LLM + Tools + Loop**: The agent uses LLM for reasoning and tools for actions
3. **Trajectory**: Complete record of agent's reasoning process

### Key differences from direct LLM:
- **Grounding**: Agent can access external information via tools
- **Iteration**: Agent can refine its approach based on observations
- **Transparency**: Trajectory shows reasoning process

### Next steps:
- Week 2: Advanced prompting (CoT, ToT)
- Week 3: Real tool integration with MCP

## 8. Extension Exercise (Optional)

Try extending the agent:
1. Add a new tool (e.g., WeatherTool, WikipediaTool)
2. Implement error handling for failed tool calls
3. Add a memory system to remember previous interactions

In [ ]:
# Your extension code here
# Example: Add a custom tool

class DateTimeTool(SimpleTool):
    """Tool to get current date/time."""
    name = "DateTime"
    description = "Get current date and time. Input: 'now'."
    
    def run(self, input_str: str) -> str:
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Add to tools
TOOLS["DateTime"] = DateTimeTool()

# Test
agent_extended = ReActAgent(tools=TOOLS)
trajectory = agent_extended.run("What is the current date?")